<a href="https://colab.research.google.com/github/yashveersinghsohi/machine_hack_competitions/blob/feature_engineering/Data_Science_Student_Championship/Features/Feature_Engineering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Git Clone

In [3]:
# !git clone -b feature_engineering https://github.com/yashveersinghsohi/machine_hack_competitions.git

# Imports

In [2]:
import pandas as pd
import numpy as np

# Datasets

In [5]:
root_dir = '/content/machine_hack_competitions/Data_Science_Student_Championship/Features/raw/'

X_train_raw = pd.read_csv(root_dir + 'X_train_raw.csv')
y_train_raw = pd.read_csv(root_dir + 'y_train_raw.csv')

X_val_raw = pd.read_csv(root_dir + 'X_val_raw.csv')
y_val_raw = pd.read_csv(root_dir + 'y_val_raw.csv')

X_test_raw = pd.read_csv(root_dir + 'X_test_raw.csv')
y_test_raw = pd.read_csv(root_dir + 'y_test_raw.csv')

In [6]:
X_train, y_train = X_train_raw.copy(), y_train_raw.copy()
X_val, y_val = X_val_raw.copy(), y_val_raw.copy()
X_test, y_test = X_test_raw.copy(), y_test_raw.copy()

# Feature Engineering